In [ ]:
import os
import csv
import tempfile
import statistics
from typing import List, Dict, Tuple

FILE_NAME = "students.txt"
FIELDNAMES = ['id', 'name', 'age', 'grade', 'marks']
VALID_GRADES = set("ABCDEF")

def safe_read_students(filename: str = FILE_NAME) -> Tuple[List[Dict], List[str]]:
    """
    Read students from CSV-style file robustly.
    Returns (students_list, errors_list).
    Each student: {'id': str, 'name': str, 'age': int, 'grade': str, 'marks': int}
    """
    students = []
    errors = []
    if not os.path.exists(filename):
        return students, errors
    try:
        with open(filename, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            for idx, row in enumerate(reader, start=1):
                if not row:
                    continue
                if len(row) != 5:
                    errors.append(f"Line {idx}: expected 5 fields, got {len(row)}")
                    continue
                sid, name, age_s, grade_s, marks_s = [c.strip() for c in row]
                # Validate parsed row; if invalid, record error and skip
                try:
                    if not sid.isdigit():
                        raise ValueError("ID not integer")
                    age = int(age_s)
                    marks = int(marks_s)
                    grade = grade_s.upper()
                    if age <= 0 or age > 120:
                        raise ValueError("Age out of range")
                    if marks < 0 or marks > 100:
                        raise ValueError("Marks out of range")
                    if len(grade) != 1 or grade not in VALID_GRADES:
                        raise ValueError("Invalid grade")
                    if name == "" or all(ch.isdigit() or ch.isspace() for ch in name):
                        raise ValueError("Invalid name")
                except Exception as e:
                    errors.append(f"Line {idx}: {e}")
                    continue
                students.append({'id': sid, 'name': name, 'age': age, 'grade': grade, 'marks': marks})
    except Exception as e:
        errors.append(f"File read error: {e}")
    return students, errors

def atomic_write_students(students: List[Dict], filename: str = FILE_NAME):
    """
    Atomically write students to filename using a temporary file + os.replace.
    """
    dirpath = os.path.dirname(os.path.abspath(filename)) or '.'
    fd, tmp_path = tempfile.mkstemp(prefix='students_', dir=dirpath, text=True)
    try:
        with os.fdopen(fd, 'w', newline='', encoding='utf-8') as tmpf:
            writer = csv.writer(tmpf)
            for s in students:
                writer.writerow([s['id'], s['name'], str(s['age']), s['grade'], str(s['marks'])])
        os.replace(tmp_path, filename)
    finally:
        if os.path.exists(tmp_path):
            try:
                os.remove(tmp_path)
            except Exception:
                pass

def id_unique_check(sid: str, students: List[Dict]) -> Tuple[bool, str]:
    if not sid.isdigit():
        return False, "ID must be an integer."
    for s in students:
        if s['id'] == sid:
            return False, "ID already exists."
    return True, ""

def validate_name(name: str) -> Tuple[bool, str]:
    if not name or name.strip() == "":
        return False, "Name cannot be empty."
    trimmed = name.strip()
    if all(ch.isdigit() or ch.isspace() for ch in trimmed):
        return False, "Name cannot be only numbers/spaces."
    return True, ""

def validate_age(age_s: str) -> Tuple[bool, str]:
    if not age_s.isdigit():
        return False, "Age must be a positive integer."
    age = int(age_s)
    if age <= 0:
        return False, "Age must be greater than 0."
    if age > 120:
        return False, "Age unrealistic (>120)."
    return True, ""

def validate_grade(grade_s: str) -> Tuple[bool, str]:
    if len(grade_s) != 1:
        return False, "Grade must be a single letter A-F."
    if grade_s.upper() not in VALID_GRADES:
        return False, "Grade must be A, B, C, D, E or F."
    return True, ""

def validate_marks(marks_s: str) -> Tuple[bool, str]:
    if not marks_s.isdigit():
        return False, "Marks must be an integer 0-100."
    m = int(marks_s)
    if not (0 <= m <= 100):
        return False, "Marks must be between 0 and 100."
    return True, ""

def prompt_input(prompt_msg: str, validator=None, allow_empty: bool = False) -> str:
    """
    Re-prompt until valid. If allow_empty True, empty string is allowed and returned.
    Validator should return (bool, message).
    """
    while True:
        val = input(prompt_msg).strip()
        if val == "" and allow_empty:
            return ""
        if validator is None:
            return val
        ok, msg = validator(val)
        if ok:
            return val
        print("  >>", msg)


def add_student():
    """
    Add a student: prompt inputs with validation, append to file.
    """
    students, _ = safe_read_students()
    print("\n--- Add Student ---")
    sid = prompt_input("Enter ID (integer): ", lambda v: id_unique_check(v, students))
    name = prompt_input("Enter Name: ", validate_name)
    age_s = prompt_input("Enter Age: ", validate_age)
    grade = prompt_input("Enter Grade (A-F): ", validate_grade).upper()
    marks_s = prompt_input("Enter Marks (0-100): ", validate_marks)
    new = {'id': sid, 'name': name, 'age': int(age_s), 'grade': grade, 'marks': int(marks_s)}
    students.append(new)
    atomic_write_students(students)
    print("Student added successfully.")

def view_students():
    """
    Display all records neatly and show total number.
    """
    students, errors = safe_read_students()
    if errors:
        print("Note: some lines were skipped due to parsing errors:")
        for e in errors:
            print("  -", e)
    if not students:
        print("No student records found.")
        return
    # Print header
    header = f"{'ID':<6} {'Name':<28} {'Age':<4} {'G':<2} {'Marks':<5}"
    print("\n--- All Students ---")
    print(header)
    print("-" * len(header))
    for s in students:
        print(f"{s['id']:<6} {s['name'][:28]:<28} {s['age']:<4} {s['grade']:<2} {s['marks']:<5}")
    print(f"\nTotal Records: {len(students)}")

def search_student():
    """
    Search by exact ID or partial name (case-insensitive).
    """
    q = input("Enter ID or partial Name to search: ").strip().lower()
    students, errors = safe_read_students()
    found = []
    for s in students:
        if s['id'] == q or q in s['name'].lower():
            found.append(s)
    if not found:
        print("No matching student found.")
        return
    print(f"\nFound {len(found)} record(s):")
    for s in found:
        print(f"ID: {s['id']}, Name: {s['name']}, Age: {s['age']}, Grade: {s['grade']}, Marks: {s['marks']}")

def update_student():
    """
    Update any field selectively for an existing student (except ID).
    Rewrites the file atomically.
    """
    students, errors = safe_read_students()
    sid = input("Enter Student ID to update: ").strip()
    idx = next((i for i, s in enumerate(students) if s['id'] == sid), None)
    if idx is None:
        print("Student ID not found.")
        return
    s = students[idx]
    print("Current record:")
    print(f"ID: {s['id']}, Name: {s['name']}, Age: {s['age']}, Grade: {s['grade']}, Marks: {s['marks']}")
    print("Press Enter to keep current value.")
    name = prompt_input("New Name: ", validate_name, allow_empty=True)
    age_s = prompt_input("New Age: ", validate_age, allow_empty=True)
    grade = prompt_input("New Grade (A-F): ", validate_grade, allow_empty=True)
    marks_s = prompt_input("New Marks (0-100): ", validate_marks, allow_empty=True)
    if name:
        students[idx]['name'] = name
    if age_s:
        students[idx]['age'] = int(age_s)
    if grade:
        students[idx]['grade'] = grade.upper()
    if marks_s:
        students[idx]['marks'] = int(marks_s)
    atomic_write_students(students)
    print("Record updated successfully.")

def delete_student():
    """
    Delete student after confirmation, rewrite file.
    """
    students, _ = safe_read_students()
    sid = input("Enter Student ID to delete: ").strip()
    idx = next((i for i, s in enumerate(students) if s['id'] == sid), None)
    if idx is None:
        print("Student ID not found.")
        return
    print("Record to delete:", students[idx])
    confirm = input("Type 'YES' to confirm deletion: ").strip()
    if confirm == "YES":
        students.pop(idx)
        atomic_write_students(students)
        print("Record deleted.")
    else:
        print("Deletion cancelled.")




def _percentile(data: List[int], p: float) -> float:
    """
    Compute percentile p (0-100) of data list (interpolated).
    """
    if not data:
        return 0.0
    data_sorted = sorted(data)
    k = (len(data_sorted) - 1) * (p / 100.0)
    f = int(k)
    c = f + 1
    if c >= len(data_sorted):
        return float(data_sorted[-1])
    d0 = data_sorted[f] * (c - k)
    d1 = data_sorted[c] * (k - f)
    return d0 + d1

def analyze_data():
    """
    Analyze records and print required + extra insights.
    Returns a summary dict if needed for export.
    """
    students, errors = safe_read_students()
    if errors:
        print("Note: some lines were skipped due to parsing errors:")
        for e in errors:
            print("  -", e)
    if not students:
        print("No data to analyze.")
        return None
    marks = [s['marks'] for s in students]
    avg = statistics.mean(marks)
    med = statistics.median(marks)
    try:
        stddev = statistics.pstdev(marks)
    except Exception:
        stddev = 0.0
    highest = max(students, key=lambda x: x['marks'])
    lowest = min(students, key=lambda x: x['marks'])
    below_avg = len([m for m in marks if m < avg])
    passed = len([m for m in marks if m >= 50])
    fail = len(marks) - passed
    pass_rate = (passed / len(marks)) * 100.0
    p25 = _percentile(marks, 25)
    p75 = _percentile(marks, 75)

    # Grade distribution and per-grade averages
    grade_buckets = {}
    for s in students:
        grade_buckets.setdefault(s['grade'], []).append(s['marks'])
    grade_summary = {g: {'count': len(lst), 'avg': statistics.mean(lst) if lst else 0.0} for g, lst in grade_buckets.items()}

    # Top performer(s) — handle ties
    top_marks = highest['marks']
    top_performers = [s for s in students if s['marks'] == top_marks]

    print("\n--- Data Analysis ---")
    print(f"Records analyzed: {len(students)}")
    print(f"Average Marks: {avg:.2f}")
    print(f"Median Marks: {med}")
    print(f"Std Dev (population): {stddev:.2f}")
    print(f"Top Performer(s): {', '.join([f'{t['name']} ({t['marks']})' for t in top_performers])}")
    print(f"Students Below Average: {below_avg}")
    print(f"Highest Marks: {highest['marks']} | Lowest Marks: {lowest['marks']}")
    print(f"Pass Count: {passed} | Fail Count: {fail} | Pass Rate: {pass_rate:.2f}%")
    print(f"25th percentile: {p25} | 75th percentile: {p75}")
    print("\nGrade Distribution:")
    for g in sorted(grade_summary.keys()):
        print(f"  Grade {g}: Count={grade_summary[g]['count']}, Avg={grade_summary[g]['avg']:.2f}")

    # Additional custom insights:
    # - Top 3 and bottom 3 lists
    top3 = sorted(students, key=lambda x: x['marks'], reverse=True)[:3]
    bottom3 = sorted(students, key=lambda x: x['marks'])[:3]
    print("\nTop 3 students:")
    for s in top3:
        print(f"  {s['name']} (ID {s['id']}) - {s['marks']}")
    print("\nBottom 3 students:")
    for s in bottom3:
        print(f"  {s['name']} (ID {s['id']}) - {s['marks']}")

    # Programmatic summary for export or tests
    summary = {
        'count': len(students), 'average': avg, 'median': med, 'stddev': stddev,
        'top_performers': top_performers, 'below_avg': below_avg,
        'highest': highest, 'lowest': lowest, 'pass_count': passed, 'fail_count': fail,
        'pass_rate': pass_rate, 'p25': p25, 'p75': p75, 'grade_summary': grade_summary,
        'top3': top3, 'bottom3': bottom3
    }
    return summary

def export_report(summary: Dict, filename: str = "students_report.txt"):
    """
    Export analysis summary to a human-readable text file.
    """
    if not summary:
        print("No summary to export.")
        return
    lines = []
    lines.append("SMART STUDENT RECORD ANALYSIS REPORT\n")
    lines.append(f"Records analyzed: {summary['count']}")
    lines.append(f"Average Marks: {summary['average']:.2f}")
    lines.append(f"Median Marks: {summary['median']}")
    lines.append(f"Std Dev: {summary['stddev']:.2f}")
    lines.append(f"Highest: {summary['highest']['name']} ({summary['highest']['marks']})")
    lines.append(f"Lowest: {summary['lowest']['name']} ({summary['lowest']['marks']})")
    lines.append(f"Pass Rate: {summary['pass_rate']:.2f}%\n")
    lines.append("Top performers:")
    for t in summary['top_performers']:
        lines.append(f"  {t['name']} (ID {t['id']}) - {t['marks']}")
    lines.append("\nGrade Summary:")
    for g, v in summary['grade_summary'].items():
        lines.append(f"  Grade {g}: Count={v['count']}, Avg={v['avg']:.2f}")
    lines.append("\nTop 3:")
    for t in summary['top3']:
        lines.append(f"  {t['name']} (ID {t['id']}) - {t['marks']}")
    lines.append("\nBottom 3:")
    for b in summary['bottom3']:
        lines.append(f"  {b['name']} (ID {b['id']}) - {b['marks']}")
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("\n".join(lines))
    print(f"Report exported to '{filename}'")

def seed_sample_data(force: bool = True):
    """
    Create sample dataset. Overwrites existing file if force True.
    """
    if os.path.exists(FILE_NAME) and not force:
        print("students.txt already exists; use force=True to overwrite.")
        return
    sample = [
        {'id': '101', 'name': 'Ali Ahmed', 'age': 17, 'grade': 'A', 'marks': 92},
        {'id': '102', 'name': 'Sana Khan', 'age': 18, 'grade': 'B', 'marks': 76},
        {'id': '103', 'name': 'Zara Ali', 'age': 16, 'grade': 'A', 'marks': 95},
        {'id': '104', 'name': 'Hassan R', 'age': 17, 'grade': 'C', 'marks': 58},
        {'id': '105', 'name': 'Ayesha M', 'age': 18, 'grade': 'B', 'marks': 69},
        {'id': '106', 'name': 'Bilal', 'age': 19, 'grade': 'D', 'marks': 44},
        {'id': '107', 'name': 'Nadia', 'age': 17, 'grade': 'A', 'marks': 88},
    ]
    atomic_write_students(sample)
    print("Sample data seeded into 'students.txt' (overwritten).")

def demo_mode():
    """
    Seed sample data, view, analyze and export a report for quick hackathon demo.
    """
    seed_sample_data(force=True)
    view_students()
    summary = analyze_data()
    export_report(summary)
    print("\nDemo complete. Open 'students_report.txt' to show judges.")


def main():
    last_summary = None
    while True:
        print("\n====== Smart Student Record Analyzer ======")
        print("1. Add Student")
        print("2. View All Students")
        print("3. Search Student")
        print("4. Update Student")
        print("5. Delete Student")
        print("6. Analyze Data")
        print("7. Export Last Analysis Report")
        print("8. Seed Sample Data (demo)")
        print("9. Exit")
        choice = input("Enter your choice (1-9): ").strip()
        if choice == '1':
            add_student()
        elif choice == '2':
            view_students()
        elif choice == '3':
            search_student()
        elif choice == '4':
            update_student()
        elif choice == '5':
            delete_student()
        elif choice == '6':
            last_summary = analyze_data()
        elif choice == '7':
            if last_summary:
                export_report(last_summary)
            else:
                print("No analysis run yet. Run option 6 first.")
        elif choice == '8':
            seed_sample_data(force=True)
        elif choice == '9':
            print("Thank you for using the Smart Student Record Analyzer. Good luck!")
            break
        else:
            print("Invalid choice. Enter 1-9.")

if __name__ == "__main__":
    main()
